## Lab 5

## 1. Run a multiple linear probability model (have at least 2 Xs in the model).  Tell me how you think your independent variables will affect your dependent variable.  Interpret your results.  Were your expectations correct?  Why or why not?

## 2. Run a multiple (binary) logistic model.  (It can be the same as the above LPM or a new model.)  If it is a new model, tell me how you think your independent variables will affect your dependent variable.  Interpret your results in the logit scale.  Were your expectations correct?  Why or why not?

## 3. Get odds ratios from your logit model in Question 2 and interpret some of them.  

## 4. Extra Credit: Get predicted probabilities from your logit model in Question 2 for some constellations of X values and interpret the results.  

In [27]:
# importing relevant modules...
from __future__ import division
import pandas as pd
import numpy as np
import re, os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns #os.system('pip install seaborn')
from scipy import stats

# defining helper functions for later in my code
def recode_onehot_if(s='CHILDRENS COAT', pattern='CHILDREN'):
    result = '1' if pattern in str(s) else '0'
    return(result)

#### Using the similar kaggle dataset again that i used in lab4 on forbes billionaires 2021. You can see the specific variables and observations in my code output below!

In [29]:
# Below, I'm automatically pulling the Kaggle dataset from the online URL.
os.system('kaggle datasets download alexanderbader/forbes-billionaires-of-2021-20/forbes_billionaires.csv')
import zipfile 
archive = zipfile.ZipFile('forbes-billionaires-of-2021-20.zip')
filename = archive.filelist[0].filename
xlfile = archive.open(filename)
df = pd.read_csv(xlfile)
os.system("rm forbes-billionaires-of-2021-20.zip")

0

In [4]:
df = df.assign(education_status = df.Education.str.replace('.*(Drop Out|Bachelor|Master|Doctor|High School).*', '\\1', regex=True),
               high_ed_grad = df.Education.apply(lambda s: 'Higher Education Grad' if s in [''] else 'No Higher Education Grad'),
               relationship_or_single = df.Status.astype(str).replace({'(Widowed$|Separated|Single|Divorced).*':'Single', '(Widowed,|In|Married|Engaged|Remarried).*':'In Relationship'}, regex=True).replace({'nan': np.nan}),
              )
print(df.Status.value_counts())
print(df.relationship_or_single.value_counts())
print(df.education_status.value_counts())
print(df.columns.tolist())
df

Married               1734
Divorced               144
Widowed                 95
Single                  74
In Relationship         21
Separated               10
Widowed, Remarried       8
Engaged                  4
Name: Status, dtype: int64
In Relationship    1767
Single              323
Name: relationship_or_single, dtype: int64
Bachelor                                                                                     789
Master                                                                                       334
Doctor                                                                                        85
Drop Out                                                                                      66
High School                                                                                   41
                                                                                            ... 
Liaoning University                                                                 

,Name,NetWorth,Country,Source,Rank,Age,Residence,Citizenship,Status,Children,Education,Self_made,education_status,high_ed_grad,relationship_or_single
0,Jeff Bezos,177.0,United States,Amazon,1,57.0,"Seattle, Washington",United States,In Relationship,4.0,"Bachelor of Arts/Science, Princeton University",True,Bachelor,No Higher Education Grad,In Relationship
1,Elon Musk,151.0,United States,"Tesla, SpaceX",2,49.0,"Austin, Texas",United States,In Relationship,7.0,"Bachelor of Arts/Science, University of Pennsy...",True,Bachelor,No Higher Education Grad,In Relationship
2,Bernard Arnault & family,150.0,France,LVMH,3,72.0,"Paris, France",France,Married,5.0,"Bachelor of Arts/Science, Ecole Polytechnique ...",False,Bachelor,No Higher Education Grad,In Relationship
3,Bill Gates,124.0,United States,Microsoft,4,65.0,"Medina, Washington",United States,Divorced,3.0,"Drop Out, Harvard University",True,Drop Out,No Higher Education Grad,Single
4,Mark Zuckerberg,97.0,United States,Facebook,5,36.0,"Palo Alto, California",United States,Married,2.0,"Drop Out, Harvard University",True,Drop Out,No Higher Education Grad,In Relationship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750,Daniel Yong Zhang,1.0,China,e-commerce,2674,49.0,"Hangzhou, China",China,NaN,NaN,NaN,True,NaN,No Higher Education Grad,NaN
2751,Zhang Yuqiang,1.0,China,Fiberglass,2674,65.0,"Tongxiang, China",China,NaN,NaN,NaN,True,NaN,No Higher Education Grad,NaN
2752,Zhao Meiguang,1.0,China,gold mining,2674,58.0,"Jilin, China",China,NaN,NaN,NaN,True,NaN,No Higher Education Grad,NaN
2753,Zhong Naixiong,1.0,China,conglomerate,2674,58.0,"Foshan, China",China,NaN,NaN,NaN,True,NaN,No Higher Education Grad,NaN


In [28]:
print("Columns in our data:\n\n", df.columns.tolist())

Columns in our data:

 ['Name', 'NetWorth', 'Country', 'Source', 'Rank', 'Age', 'Residence', 'Citizenship', 'Status', 'Children', 'Education', 'Self_made', 'education_status', 'high_ed_grad', 'relationship_or_single', 'relationship_indicator']


# 1. Run a multiple linear probability model (have at least 2 Xs in the model).  Tell me how you think your independent variables will affect your dependent variable.  Interpret your results.  Were your expectations correct?  Why or why not?

### The question is what are some things that predict how likely someone is to be in a romantic relationship.

### I am going to recode to only look at if someone is "in a relationship" vs. everything else (AKA single)

<!-- Dependent Variables: Net Worth 
Independent Variables: Age, Country  -->

<b>Dependent Variables:</b> <code>Probability of...being in a relationship or single</code>
<b>Independent Variables:</b> <code>Age, Self_made</code>

<b>Hypothesis/expectations:</b>
I expect that those who have started their own companies and made their wealth by themselves (not inherited from generations above), are more likely to be in relationships, regardless of their age. This seems to make theoretical sense to me because self-made folks probably have a uniquely good work ethic that might extend to their personal lives and the support of romantic partners might go hand in hand with their professional successes.

Below, I'm recoding the variable about relationship status as 1=In a Relationship versus 0=Not in a relationship (totally single)

In [26]:
conditions = [
    (df['relationship_or_single'] == 'In Relationship') ,
    (df['relationship_or_single'] == 'Single'),
]
choices = [1, 0]
df['relationship_indicator'] = np.select(conditions, choices, default=np.nan)
pd.crosstab(index=df["relationship_indicator"], columns="count")  ## check that the recode worked okay (just like Eirich did in his sample lab)

col_0,count
relationship_indicator,
0.0,323
1.0,1767


### I'm subsetting the LPM on only people in the United States, just to make it more specific to the businesspeople of the country we're living in.

In [7]:
lm1 = smf.ols(formula = 'relationship_indicator ~ Age + Self_made', subset=(df['Country']=='United States'), data = df).fit()
print (lm1.summary())

                              OLS Regression Results                              
Dep. Variable:     relationship_indicator   R-squared:                       0.037
Model:                                OLS   Adj. R-squared:                  0.034
Method:                     Least Squares   F-statistic:                     12.32
Date:                    Sat, 12 Jun 2021   Prob (F-statistic):           5.61e-06
Time:                            22:35:13   Log-Likelihood:                -309.20
No. Observations:                     653   AIC:                             624.4
Df Residuals:                         650   BIC:                             637.8
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

### These results make sense to me.  Those who are self-made are more likely to be in a relationship (on average) by .1608 units, net of their age. To build large businesses requires more than just an incredible work ethic. It requires a support network professionally and personally. This logical reasoning is supported by the statistically significant relationship, with the p-value being 0.000 (b = 0.1608, p<0.001)

# 2. Run a multiple (binary) logistic model.  (It can be the same as the above LPM or a new model.)  If it is a new model, tell me how you think your independent variables will affect your dependent variable.  Interpret your results in the logit scale.  Were your expectations correct?  Why or why not?


In [8]:
logit1 = sm.formula.logit(formula = 'relationship_indicator ~ Age + Self_made', subset=(df['Country']=='United States'), data = df).fit()
print (logit1.summary())

Optimization terminated successfully.
         Current function value: 0.475456
         Iterations 6
                             Logit Regression Results                             
Dep. Variable:     relationship_indicator   No. Observations:                  653
Model:                              Logit   Df Residuals:                      650
Method:                               MLE   Df Model:                            2
Date:                    Sat, 12 Jun 2021   Pseudo R-squ.:                 0.03493
Time:                            22:35:14   Log-Likelihood:                -310.47
converged:                           True   LL-Null:                       -321.71
Covariance Type:                nonrobust   LLR p-value:                 1.318e-05
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             1.3812      0.559      2.471      0.013 

### Above are the logit coefficients; In the United States, if the individual is self-made, on average, there is a higher logit of being in a relationship.
### For billionaires who are self-made (as opposed to wealthy through inheritance) statistically significantly increases her/his logit by 0.9369 of being in a relationship on average, net of age (p < .001). This is pretty much in line with what I imagined; however, I somewhat expected older people to be more likely to be in a relationship. As it turns out, there is no statistically significant effect of Age on our dependent variable.


# 3. Get odds ratios from your logit model in Question 2 and interpret some of them.  


In [11]:
np.exp(logit1.params)

Intercept            3.979749
Self_made[T.True]    2.552157
Age                  0.991459
dtype: float64

### For the odds-ratios above, for the individual that is self-made, the odds that she/he is in a relationship goes up by 155%, net of other factors (age).
### For each year older an individual is in age, the odds that he/she is in a relationship goes down by .8% (0.991459-1=-.008541), net of other factors (being self made), so barely at all.



# 4. Extra Credit: Get predicted probabilities from your logit model in Question 2 for some constellations of X values and interpret the results.  

In [17]:
def logit2prob (logit): # Credit to Eirich for this function!
    odds = np.exp(logit)
    prob = odds / (1 + odds) 
    return(prob);

intercept = logit1.params.Intercept
b_selfmade = logit1.params['Self_made[T.True]']
b_age = logit1.params.Age

## I am going to find out the predicted probability of being in a relationship for a person who is self made (as I hope to be!) and is 23 years old (same age as me!).

In [19]:
## CHOOSE REPRESENTATIVE VALUES FOR ALL Xs ##
logits_exh = intercept + (1 * b_selfmade) + (23 * b_age)
logit2prob(logits_exh)

0.8929150906180534

## A 23-year old self-made billionaire in the US has a 89.29 probability of being in a romantic relationship. Seems high, but not ridiculous!

## I am also going to find out the predicted probability of being in a relationship for a person who is NOT self made and is 23 years old (same age as me!).

In [20]:
## CHOOSE REPRESENTATIVE VALUES FOR ALL Xs ##
logits_exh = intercept + (0 * b_selfmade) + (30 * b_age)
logit2prob(logits_exh) 

0.7547088445205984

## A 23-year old billionaire who inherited his/her wealth in the US has a 75.47 probability of being in a romantic relationship. A bit lower!

## Additionally, I am going to find out the predicted probability of being in a relationship for a person who has average values on the Age X, which happens to be 63.27 years-old and is self-made.

In [23]:
df.Age.mean()

63.267300380228136

In [21]:
logits_exh = intercept + (1 * b_selfmade) + (df.Age.mean() * b_age)
logit2prob(logits_exh) 

0.8551361423018715

## Huh! A lower probability than that of the 23 year old!? An average-aged billionaire in the US has a 85.51 probability of being in a romantic relationship. This surprises me because I'd image someone of 63 years would be more likely to have settled down in a relationship than a post-adolescent 23-year-old novice like me!

**Disclaimer! I am not _yet_ a billionaire haha